In [1]:
from pathlib import Path

import pint
from util import p_

import autochem
import automech
from autochem.util import plot

kB = pint.Quantity("boltzmann_constant").m_as("J/K")


class Key:
    allyl = "allyl"
    alkyl = "alkyl"
    vinyl = "vinyl"


eq_dct = {
    Key.allyl: "C5H8(522) + OH(4) = C5H7(500) + H2O(5)",
    Key.alkyl: "C5H8(522) + OH(4) = C5H7(504) + H2O(5)",
    Key.vinyl: "C5H8(522) + OH(4) = C5H7(1202) + H2O(5)",
}

exp_data_dct = {
    Key.allyl: {
        "order": 2,
        "A": 2.55e-11,
        "b": 0,
        "E": 1560 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
    Key.alkyl: {
        "order": 2,
        "A": 1.90e-11,
        "b": 0,
        "E": 1705 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
    Key.vinyl: {
        "order": 2,
        "A": 1.39e-11,
        "b": 0,
        "E": 2270 * kB,
        "units": {"substance": "molec", "energy": "J"},
    },
}

tag = "A_rh-oh_1e-1_v0"
root_path = Path("..") / Path("..")

In [2]:
par_mech = automech.io.read(p_.parent_mechanism("json", path=p_.data(root_path)))
cal_mech = automech.io.read(
    p_.calculated_mechanism(tag, "json", path=p_.data(root_path))
)

In [3]:
for key in (Key.allyl, Key.alkyl, Key.vinyl):
    print(key)

    cal_rxns = automech.reaction_rate_objects(cal_mech, eq_dct.get(key))
    par_rxns = automech.reaction_rate_objects(par_mech, eq_dct.get(key))
    exp_rate = autochem.rate.data.ArrheniusRateFit.model_validate(exp_data_dct.get(key))

    assert len(cal_rxns) == 1, cal_rxns
    (cal_rxn,) = cal_rxns
    ks = [cal_rxn.rate, exp_rate]
    labels = ["This work", "KAUST"]
    colors = [plot.Color.blue, plot.Color.brown]

    if par_rxns:
        assert len(par_rxns) == 1, par_rxns
        (par_rxn,) = par_rxns
        ks.append(par_rxn.rate)
        labels.append("Hill")
        colors.append(plot.Color.red)

    chart = autochem.util.plot.arrhenius(
        ks=ks, labels=labels, order=cal_rxn.rate.order, T_range=(856, 1258)
    )

    display(chart)

    chart.save(f"{key}.png", ppi=300)

allyl


alt.Chart(...)

alkyl


alt.Chart(...)

vinyl


alt.Chart(...)